<a href="https://colab.research.google.com/github/FeraMaks/test/blob/main/NN_ShaftTraectory_2_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader
import numpy as np
import matplotlib.pyplot as plt
import math
import random
import pandas as pd
from numpy.core.memmap import uint8

from google.colab import drive

In [2]:
#device config
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [3]:
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


##Загрузка данных

In [4]:
# Советую воспользоваться циклом for, чтобы не писать множество однотипных строк кода

data1_x = pd.read_excel(open('/content/drive/MyDrive/work/100.xlsx', 'rb'))
data1_x = pd.DataFrame(data1_x)
data1_xnp = pd.DataFrame.to_numpy(data1_x)
data1_xnp = np.rot90(data1_xnp)
data1_xnp.shape

(6, 20000)

In [5]:
data2_x = pd.read_excel(open('/content/drive/MyDrive/work/90_011.xlsx', 'rb'))
data2_x = pd.DataFrame(data2_x)
data2_xnp = pd.DataFrame.to_numpy(data2_x)
data2_xnp = np.rot90(data2_xnp)
data2_xnp.shape

(6, 20000)

In [6]:
data3_x = pd.read_excel(open('/content/drive/MyDrive/work/110_011.xlsx', 'rb'))
data3_x = pd.DataFrame(data3_x)
data3_xnp = pd.DataFrame.to_numpy(data3_x)
data3_xnp = np.rot90(data3_xnp)
data3_xnp.shape

(6, 20000)

In [7]:
data4_x = pd.read_excel(open('/content/drive/MyDrive/work/120_011.xlsx', 'rb'))
data4_x = pd.DataFrame(data4_x)
data4_xnp = pd.DataFrame.to_numpy(data4_x)
data4_xnp = np.rot90(data4_xnp)
data4_xnp.shape

(6, 20000)

##Подготовка датасета

In [8]:
#parameters
data_number = 4 # количество файлов с графиками
num_samples = 10 # количество сэмплов в одном графике
point_number = 2000 # количество точек в одном графике
num_graphycs = 3 # количество графиков
num_grahp0 = 1 # кол-во нормальных графиков
num_grahp1 = 2 # кол-во угловатых графиков
inputs = []

In [9]:
y0 = np.zeros((num_grahp0*num_samples,1))
y1 = np.ones((num_grahp1*num_samples,1))
y = np.concatenate((y0, y1))
for i in range (2):
  y = np.concatenate((y, y))
y, y.shape

(array([[0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [1.],
      

In [10]:
data_x = np.concatenate((data1_xnp,data2_xnp,data3_xnp,data4_xnp))
type(data_x)

numpy.ndarray

In [11]:
for j in range (data_number*num_graphycs):
  for i in range (num_samples):
    inputs.append(np.concatenate((data_x[j*2,i*point_number:i*point_number+point_number], data_x[j*2+1,i*point_number:i*point_number+point_number])))
inputs = np.concatenate(inputs, axis=0)
inputs = np.reshape(inputs, (data_number*num_graphycs*num_samples,point_number*2))
# Тип данных  в нейросетях - float32. Можно и float64, но тогда модель нужно делать с таким типом данных.
#x = x.astype('float32')
inputs.shape

(120, 4000)

In [12]:
xy = np.concatenate((y,inputs), axis=1)
xy[0,:]

array([   0.      , -182.154261, -182.813766, ...,  -46.86475 ,
        -44.582413,  -42.288928])

In [13]:
xy = sorted(xy, key=lambda x: random.random())

In [14]:
xy = np.asarray(xy)
xy.shape

(120, 4001)

In [15]:
x, y = xy[:,1:], xy[:,0]
x = x.astype('float32')
y = y.astype('int64')
x.shape, y.shape

((120, 4000), (120,))

In [16]:
# Индексация классов начиается с нуля. То есть первый класс - 0, второй класс - 1. Тип данных разметки - int64.
# Размерность у разметки должна быть одна

In [17]:
# Тут и далее уже очень много ошибок, покажу просто как правильно с комментариями.

class ShaftDataset(Dataset):
    def __init__(self, x, y, mode):
        # Проверка выборки
        if mode == 'train':
            self.x = x[:100]
            self.y = y[:100]
        elif mode == 'val':
            self.x = x[100:]
            self.y = y[100:]
            
        # Разбиение сделано по вашему примеру, но обращаю внимание,
        # что так в тренировочной выборке всего один пример со 2 классом. А в тестовой - только 2 класс/
        # Поэтому нужно как то примеры скомпоновать по-другому

    def __getitem__(self, index):
        return self.x[index], self.y[index]
  
    def __len__(self):
        return len(self.y)

In [18]:
# Создаем два отдельных датасета и их будем подавать в Loader.
# Изначально вы пытались сделать два среза по одному датасету и подавили в Loader. Так работать не будет

train_dataset = ShaftDataset(x, y, 'train')
test_dataset = ShaftDataset(x, y, 'val')


In [19]:
train_dataset[0]

(array([-312.547  , -317.00662, -321.39444, ..., -386.78918, -383.58664,
        -380.30704], dtype=float32), 1)

Hyper parameters

In [20]:
input_size = x.shape[1]
hidden_size = 32 # Попробуйте для начала поменьше
num_classes = 2
num_epochs = 10000
batch_size = 15 # Bath size не может быть больше размера датасета (у вас 30 примеров)
lr = 0.001 # learning rate

In [21]:
# Loader генерирует тензоры следующего размера -
# (batch_size (размер батча - кол-во примеров в батче), m (кол-во признаков))
# Таким образом, размерностей 2, а не 4

train_loader = torch.utils.data.DataLoader(dataset=train_dataset, 
                                           batch_size=batch_size, 
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset, 
                                          batch_size=batch_size, 
                                          shuffle=False)

In [22]:
# Размеры данных в датасетах

features, labels = train_dataset.x, train_dataset.y
print(features.shape, labels.shape)
features, labels = test_dataset.x, test_dataset.y
print(features.shape, labels.shape)
#samples = torch.from_numpy(samples)
#labels = torch.from_numpy(labels)

(100, 4000) (100,)
(20, 4000) (20,)


#Создание нейронной сети

In [23]:
class NeuralNet(nn.Module):
  def __init__(self, input_size, hidden_size, num_classes):
    super(NeuralNet, self).__init__()
    self.l1 = nn.Linear(input_size, hidden_size)
    self.relu = nn.ReLU()
    self.l2 = nn.Linear(hidden_size, num_classes)

  def forward(self, x):
    out = self.l1(x)
    out = self.relu(out)
    out = self.l2(out)
    return out


In [24]:
model = NeuralNet(input_size, hidden_size, num_classes)

# Модель также перемещаем в нужное место
model = model.to(device)

Loss and optimizer

In [25]:
L = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=lr)

#Training loop

In [ ]:
n_total_steps = len(train_loader)
for epoch in range(num_epochs):
  for i, (features, labels) in enumerate(train_loader):
    features = features.to(device)
    labels = labels.to(device)

    #forward path
    outputs = model(features) # Подаем признаки на вход нейросети
    loss = L(outputs, labels)

    #backward path
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if (i+1) % 7 == 0:
       print(f'epoch {epoch+1} / {num_epochs}, step {i+1}/{n_total_steps}, loss = {loss.item():.4f}')


In [27]:
outputs, labels

(tensor([[  23.2449,    7.2273],
         [1465.5220, 1492.9529],
         [  -2.7491,  -15.6126],
         [ 126.7009,  146.1419],
         [ 375.8835,  537.9810],
         [1907.5239, 1929.6212],
         [ 696.1847,  721.3726],
         [ 208.8569,  352.0326],
         [  70.5079,   92.2346],
         [ 168.0563,  150.4929]], grad_fn=<AddmmBackward0>),
 tensor([0, 1, 0, 1, 1, 1, 1, 1, 1, 0]))

#Test

In [28]:
with torch.no_grad():
  n_correct = 0
  n_samples = 0
  for features, labels in test_loader:
    features = features.to(device)
    labels = labels.to(device)
    outputs = model(features)
    print(outputs)
    #value, index
    _, predictions = torch.max(outputs, 1)
    n_samples += labels.shape[0]
    n_correct += (predictions == labels).sum().item()

tensor([[ 385.8858,  399.5749],
        [ 262.2820,  279.7303],
        [ 750.1978,  758.6814],
        [1239.1327, 1229.9988],
        [ 408.8623,  396.8980],
        [ 553.9085,  509.5866],
        [ 312.2325,  329.7563],
        [ 129.1398,  129.8865],
        [ 438.5577,  467.7952],
        [ 195.6100,  219.9076],
        [1860.6306, 1881.1155],
        [ 167.2228,  161.4828],
        [ 136.7680,  139.8704],
        [ 217.8830,  233.7505],
        [ 193.5891,  216.8350]])
tensor([[ 710.3569,  699.3018],
        [ 208.5720,  206.6598],
        [ 394.9173,  505.9138],
        [1597.5796, 1644.1211],
        [  68.7338,   90.4059]])


In [29]:
  acc = 100.0 * n_correct / n_samples
  print(f'accuracy = {acc} %')

accuracy = 60.0 %
